In [1]:
def save(triple_set, file_):
    out_str = ""
    for triple in triple_set:
        h, r, t = triple
        if h != None and r != None and t != None:
            out_str += h + '\t' + r + '\t' + t +'\n'

    with open(file_, 'w') as f:
        f.write(out_str)

In [2]:
import requests
from tqdm import tqdm

import re

def retrive_value(response):
    # assuming the HTML response is stored in the variable 'html'
    has_value_pattern = r'<span class="value">([\d.]+)</span>'
    has_value_match = re.search(has_value_pattern, response)

    if has_value_match:
        value = has_value_match.group(1)
        return value
    else:
        return None

with open('../compound_descriptor.txt', 'r') as f:
    lines = f.readlines()

rel_set = set()
triple_set = set()
for i in tqdm(range(len(lines[850000:]))):
    h, r, t = lines[i].split('\t')
    t = t[:-1]
    t_query = t.replace('descriptor:', '')
    rel = t_query.replace(t_query.split('_')[0]+'_', '')
    rel_set.add(rel)

    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/rdf/descriptor/{t_query}.html"

    # Query the API and parse the response
    try:
        response = requests.get(url).text
    except:
        continue

    value = retrive_value(response)

    triple = (h, rel, value)
    triple_set.add(triple)

    if i % 10000 == 0 and i != 0:
        save(triple_set=triple_set, file_='../compound_descriptor_triple_6.txt')


100%|██████████| 188933/188933 [8:30:59<00:00,  6.16it/s]   


In [3]:
save(triple_set=triple_set, file_='../compound_descriptor_triple_6.txt')